In [39]:
# IMPORT
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

Build a regression model.

In [40]:
# PULL PREVIOUS DATA FRAME THROUGH SQLITE3 DATABASE
connection = sqlite3.connect("sqldata.sqlite")
sql_df = pd.read_sql_query('SELECT * FROM merged_df2', connection)
connection.close()

In [41]:
sql_df.head()

,index,name_x,latitude,longitude,free_bikes,fsq_id,category,distance,country,city,address,name_y,name,review_count,rating,price,catergory
0,0,André Mazet - Saint-André des Arts,48.85,2.34,23,4bbde5128a4fb713b57e3d9d,Tea Room,52,FR,Paris,59-61 rue Saint-André des Arts,La Jacobine,L'Avant Comptoir,643,4.5,€€,Tapas Bars
1,1,André Mazet - Saint-André des Arts,48.85,2.34,23,4bbde5128a4fb713b57e3d9d,Tea Room,52,FR,Paris,59-61 rue Saint-André des Arts,La Jacobine,Bistro des Augustins,463,4.5,€€,Bistros
2,2,André Mazet - Saint-André des Arts,48.85,2.34,23,4bbde5128a4fb713b57e3d9d,Tea Room,52,FR,Paris,59-61 rue Saint-André des Arts,La Jacobine,La Jacobine,313,4.5,€€€,Tea Rooms
3,3,André Mazet - Saint-André des Arts,48.85,2.34,23,4bbde5128a4fb713b57e3d9d,Tea Room,52,FR,Paris,59-61 rue Saint-André des Arts,La Jacobine,Jardin du Luxembourg,380,4.5,None,Parks
4,4,André Mazet - Saint-André des Arts,48.85,2.34,23,4bbde5128a4fb713b57e3d9d,Tea Room,52,FR,Paris,59-61 rue Saint-André des Arts,La Jacobine,Little Breizh,209,4.5,€€,Creperies


In [42]:
sql_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11945 entries, 0 to 11944
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         11945 non-null  int64  
 1   name_x        11945 non-null  object 
 2   latitude      11945 non-null  float64
 3   longitude     11945 non-null  float64
 4   free_bikes    11945 non-null  int64  
 5   fsq_id        11945 non-null  object 
 6   category      11945 non-null  object 
 7   distance      11945 non-null  int64  
 8   country       11945 non-null  object 
 9   city          11945 non-null  object 
 10  address       11945 non-null  object 
 11  name_y        11945 non-null  object 
 12  name          11945 non-null  object 
 13  review_count  11945 non-null  int64  
 14  rating        11945 non-null  float64
 15  price         10521 non-null  object 
 16  catergory     11945 non-null  object 
dtypes: float64(3), int64(4), object(10)
memory usage: 1.5+ MB


In [43]:
sql_df.describe()

,index,latitude,longitude,free_bikes,distance,review_count,rating
count,11945.000000,11945.000000,11945.000000,11945.000000,11945.000000,11945.000000,11945.000000
mean,5972.000000,48.854369,2.343745,17.943072,430.556718,388.850984,4.417288
std,3448.368817,0.009135,0.017191,10.941249,328.978593,303.415392,0.216295
min,0.000000,48.830000,2.270000,0.000000,52.000000,67.000000,3.500000
25%,2986.000000,48.850000,2.340000,10.000000,229.000000,209.000000,4.500000
50%,5972.000000,48.850000,2.340000,19.000000,352.000000,314.000000,4.500000
75%,8958.000000,48.850000,2.340000,25.000000,424.000000,463.000000,4.500000
max,11944.000000,48.890000,2.390000,44.000000,1596.000000,2039.000000,5.000000


In [44]:
sql_df = sql_df.drop(['category','name','name_y','address', 'city', 'country','catergory','fsq_id','name_x','price','index'], axis=1) #DROPPED ALL OBJECT TYPE COLUMNS

In [45]:
print(sql_df)

       latitude  longitude  free_bikes  distance  review_count  rating
0         48.85       2.34          23        52           643     4.5
1         48.85       2.34          23        52           463     4.5
2         48.85       2.34          23        52           313     4.5
3         48.85       2.34          23        52           380     4.5
4         48.85       2.34          23        52           209     4.5
...         ...        ...         ...       ...           ...     ...
11940     48.86       2.36          17       374          1906     4.5
11941     48.86       2.36          17       374           218     4.5
11942     48.86       2.36          17       374           459     4.0
11943     48.86       2.36          17       374          1906     4.5
11944     48.86       2.36          17       374          1906     4.5

[11945 rows x 6 columns]


In [46]:
np.asarray(sql_df)

array([[  48.85,    2.34,   23.  ,   52.  ,  643.  ,    4.5 ],
       [  48.85,    2.34,   23.  ,   52.  ,  463.  ,    4.5 ],
       [  48.85,    2.34,   23.  ,   52.  ,  313.  ,    4.5 ],
       ...,
       [  48.86,    2.36,   17.  ,  374.  ,  459.  ,    4.  ],
       [  48.86,    2.36,   17.  ,  374.  , 1906.  ,    4.5 ],
       [  48.86,    2.36,   17.  ,  374.  , 1906.  ,    4.5 ]])

Provide model output and an interpretation of the results. 

In [49]:
# DEPENDENT & INDEPENDENT VARIABLES
y = sql_df['free_bikes']
X = sql_df.drop(['free_bikes'], axis=1)
X = sm.add_constant(X) # ADD CONSTANT

# LINEAR REGRESSION MODEL
model = sm.OLS(y, X)

# FIT MODEL
results = model.fit()

# PRINT SUMMARY
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     571.9
Date:                Mon, 27 Feb 2023   Prob (F-statistic):               0.00
Time:                        16:09:49   Log-Likelihood:                -44245.
No. Observations:               11945   AIC:                         8.850e+04
Df Residuals:                   11939   BIC:                         8.855e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         2.087e+04    510.904     40.839   

This model is supposed to show how the independent variables effect on the amount of bikes available by the dependent variable which is free_bikes.
There is a correlation between the dependent variable 'free_bikes' and the independent variables ( latitude, longitude, distance, review_count, rating). The R-squared value of 0.193 shows that 19.3% of the variance in the dependent variable can be explained by the independent variables. The p-values show that all of the variables, except for rating, are significant in predicting the number of free bikes in a location.

# Stretch

How can you turn the regression model into a classification model?